In [ ]:
import tensorflow as tf

from tensorflow import keras

In [ ]:
def data_augmentation():
    data_augmentation = keras.models.Sequential()
    data_augmentation.add(keras.layers.RandomFlip('horizontal'))
    data_augmentation.add(keras.layers.RandomRotation(0.2))
    data_augmentation.add(keras.layers.RandomZoom(0.2))
    data_augmentation.add(keras.layers.RandomContrast(0.2))



    return data_augmentation

In [ ]:
def bloco_identidade(X, f, filters, initializer=keras.initializers.random_uniform):
    F1, F2, F3 = filters

    X_shortcut = X

    X = keras.layers.Conv2D(filters=F1, kernel_size=1, strides=(1,1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = keras.layers.BatchNormalization(axis=3)(X)
    X = keras.layers.Activation('relu')(X)

    X = keras.layers.Conv2D(filters=F2, kernel_size=(f, f), strides=(1,1), padding='same', kernel_initializer=initializer(seed=0))(X)
    X = keras.layers.BatchNormalization(axis=3)(X)
    X = keras.layers.Activation('relu')(X)

    X = keras.layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(1,1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = keras.layers.BatchNormalization(axis=3)(X)

    X = keras.layers.Add()([X, X_shortcut])
    X = keras.layers.Activation('relu')(X)

    return X


In [ ]:
def bloco_convolucional(X, f, filters, s=2, initializer=keras.initializers.glorot_uniform):
    F1, F2, F3 = filters
    X_shortcut = X

    X = keras.layers.Conv2D(filters=F1, kernel_size=1, strides=(s, s), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = keras.layers.BatchNormalization(axis=3)(X)
    X = keras.layers.Activation('relu')(X)

    X = keras.layers.Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', kernel_initializer=initializer(seed=0))(X)
    X = keras.layers.BatchNormalization(axis=3)(X)
    X = keras.layers.Activation('relu')(X)

    X = keras.layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = keras.layers.BatchNormalization(axis=3)(X)

    X_shortcut = keras.layers.Conv2D(F3, (1, 1), strides=(s, s), kernel_initializer=initializer(seed=0))(X_shortcut)
    X_shortcut = keras.layers.BatchNormalization(axis=3)(X_shortcut)

    X = keras.layers.Add()([X, X_shortcut])
    X = keras.layers.Activation('relu')(X)

    return X


In [ ]:
def ResNet50(input_shape=(64, 64, 3), classes=6, training=False):
    X_input = keras.layers.Input(input_shape)
    X_input = data_augmentation()(X_input)
    X = keras.layers.ZeroPadding2D((3, 3))(X_input)

    X = keras.layers.Conv2D(64, (7, 7), strides=(2, 2), kernel_initializer=keras.initializers.glorot_uniform(seed=0))(X)
    X = keras.layers.BatchNormalization(axis=3)(X)
    X = keras.layers.Activation('relu')(X)
    X = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = bloco_convolucional(X, f=3, filters=[64, 64, 256], s=1)
    X = bloco_identidade(X, 3, [64, 64, 256])
    X = bloco_identidade(X, 3, [64, 64, 256])

    X = bloco_convolucional(X, f=3, filters=[128, 128, 512], s=2)
    X = bloco_identidade(X, f=3, filters=[128, 128, 512])
    X = bloco_identidade(X, f=3, filters=[128, 128, 512])
    X = bloco_identidade(X, f=3, filters=[128, 128, 512])

    X = bloco_convolucional(X, f=3, filters=[256, 256, 1024], s=2)
    X = bloco_identidade(X, f=3, filters=[256, 256, 1024])
    X = bloco_identidade(X, f=3, filters=[256, 256, 1024])
    X = bloco_identidade(X, f=3, filters=[256, 256, 1024])
    X = bloco_identidade(X, f=3, filters=[256, 256, 1024])
    X = bloco_identidade(X, f=3, filters=[256, 256, 1024])

    X = bloco_convolucional(X, f=3, filters=[512, 512, 2048], s=2)
    X = bloco_identidade(X, f=3, filters=[512, 512, 2048])
    X = bloco_identidade(X, f=3, filters=[512, 512, 2048])

    X = keras.layers.AveragePooling2D((2, 2))(X)

    X = keras.layers.Flatten()(X)
    X = keras.layers.Dropout(0.5)(X)
    X = keras.layers.Dense(classes, activation='softmax', kernel_initializer=keras.initializers.glorot_uniform(seed=0))(X)

    model = keras.Model(inputs=X_input, outputs=X)

    return model


In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

batch_size = 64
print(test_images.shape)
train_images.shape = (60000,28,28,1)
test_images.shape = (10000,28,28,1)
def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (64, 64))
    return image, label
X_train = train_images.astype('float32') / 255.0
X_test = test_images.astype('float32') / 255.0

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

train_dataset = (train_dataset
                 .map(lambda x, y: preprocess(x, y))
                 .shuffle(buffer_size=10000)
                 .batch(batch_size)
                 .prefetch(1))
test_dataset = (test_dataset
                .map(lambda x, y: preprocess(x, y))
                .batch(batch_size)
                .prefetch(1))

(10000, 28, 28)


In [ ]:
model = ResNet50(input_shape=(64, 64, 1), classes=10)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01,weight_decay=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath='best_model.weights.h5',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    save_weights_only=True,
    verbose=1
)

In [ ]:
model.fit(train_dataset, epochs=10, validation_data=test_dataset,callbacks=[checkpoint])

Epoch 1/10
937/938 [============================>.] - ETA: 0s - loss: 0.0630 - accuracy: 0.9826
Epoch 1: val_loss did not improve from 0.03739
938/938 [==============================] - 42s 45ms/step - loss: 0.0630 - accuracy: 0.9826 - val_loss: 0.0562 - val_accuracy: 0.9860
Epoch 2/10
938/938 [==============================] - ETA: 0s - loss: 0.0596 - accuracy: 0.9843
Epoch 2: val_loss did not improve from 0.03739
938/938 [==============================] - 42s 45ms/step - loss: 0.0596 - accuracy: 0.9843 - val_loss: 0.0982 - val_accuracy: 0.9719
Epoch 3/10
937/938 [============================>.] - ETA: 0s - loss: 0.0800 - accuracy: 0.9791
Epoch 3: val_loss did not improve from 0.03739
938/938 [==============================] - 43s 46ms/step - loss: 0.0800 - accuracy: 0.9791 - val_loss: 10.3294 - val_accuracy: 0.8198
Epoch 4/10
938/938 [==============================] - ETA: 0s - loss: 0.0604 - accuracy: 0.9849
Epoch 4: val_loss improved from 0.03739 to 0.03321, saving model to best_mo

In [ ]:
model.load_weights("best_model.weights.h5")

In [ ]:
print(f"Accuracy: {round(model.evaluate(test_dataset)[1],5)*100}%")

157/157 [==============================] - 2s 11ms/step - loss: 0.0332 - accuracy: 0.9898
Accuracy: 98.98%
